<h1>Simple Recommender</h1>
A simple recommender that is based on the popularity/ratings of movies.

The popularity-based recommender system will perform the following steps:
1. Retrieve user, item and activity data
2. Build the simple recommender
3. Generate recommendations

<h2>Step 1 - Retrieve Data</h2>
The first step would always be to gather the data and pull it into the programming environment.

For our use case, we download the MovieLens dataset containing three sets of data,
 - Movie data containing a certain movie's information, such as movieID, release date, URL, genre details, and so on
 - User data containing the user information, such as userID, age, gender, occupation, ZIP code, and so on
 - Ratings data containing userID, itemID, rating, timestamp

In [ ]:
# Import the libraries that are going to be used here
import pandas as pd
import numpy as np
import scipy
import sklearn

In [ ]:
# Column headers for the dataset
data_cols = ['user id','movie id','rating','timestamp']
item_cols = ['movie id','movie title','release date', 'video release date','IMDb URL','unknown','Action', 'Adventure','Animation','Childrens','Comedy','Crime', 'Documentary','Drama','Fantasy','Film-Noir','Horror', 'Musical','Mystery','Romance ','Sci-Fi','Thriller', 'War' ,'Western']
user_cols = ['user id','age','gender','occupation', 'zip code']

In [ ]:
# List of users
df_u_user = pd.read_csv('/home/nbuser/library/dataset/u.user', header=None, sep='|', names=user_cols, encoding='latin-1')
df_u_user = df_u_user.sort_values('user id', ascending=1)
df_u_user.columns
df_u_user.head(10)

In [ ]:
# List of movie items
df_u_item = pd.read_csv('/home/nbuser/library/dataset/u.item', header=None, sep='|', names=item_cols, encoding='latin-1')
df_u_item = df_u_item.sort_values('movie id', ascending=1)
df_u_item.columns
df_u_item.head(10)

In [ ]:
# User activity data
df_u_data = pd.read_csv('/home/nbuser/library/dataset/u.data', header=None, sep='\t', names=data_cols, encoding='latin-1')
df_u_data = df_u_data.sort_values('user id', ascending=1)
df_u_data.columns
df_u_data.head(10)

<h2>Step 2 - Building the Simple Recommender</h2>

Merge the three dataframes into one single dataframe. This allows the depiction of all the transactional activity in one single dataframe, leading to better and faster analysis.

In [ ]:
#Create one data frame from the three
dataset = pd.merge(pd.merge(df_u_item, df_u_data), df_u_user)
dataset.head(10)

Use groupby to group the movies by their titles, and the size function to returns the total number of entries under each movie title. This will help us get the number of people who rated the movie/ the number of ratings.



In [ ]:
# Get total rating count by movie
ratings_total = dataset.groupby('movie title').size()

# Convert the Series ratings_total type into a dataframe
ratings_total = pd.DataFrame({'movie title':ratings_total.index, 'total ratings': ratings_total.values})
ratings_total.head(10)

Take the mean ratings of each movie using the mean function. First we groupby movie title. From the resulting dataframe we select only the movie title and the rating headers. Then we use the mean function on them.

In [ ]:
# Get the mean rating for each movie
ratings_mean = (dataset.groupby('movie title'))['movie title','rating'].mean()

# In the previous operation, movie title has been converted from a column to an index. So we make that a column again
ratings_mean['movie title'] = ratings_mean.index

ratings_mean.head(10)

Merge the two dataframes (containing mean ratings and total ratings) together. While doing the merge, sort the values by the total rating (i.e. based on popularity of the movie).

In [ ]:
final = pd.merge(ratings_mean, ratings_total).sort_values(by = 'total ratings', ascending= False)
final.head(10)

We need to look at the basic characteristics of the data to determine the minimum cutoff of total ratings. Because its not reliable to recommend a movie with a high mean rating that has been rated by only 10 people.

In [ ]:
print(final.describe())

<h2>Step 3 - Generating Recommendations</h2>

For generating recommendations, we look at the most popular movies. These can be based on the
- total ratings for a movie
- average rating of a movie

In [ ]:
final = final[:300].sort_values(by = 'rating', ascending = False)
print(final.head())